We employ FlauBERT to enhance the performance of our natural language processing task. To further boost the model's proficiency, we augment our dataset. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
# Read the CSV file
training_data = pd.read_csv("../Dataset_upgrade/training_dataUP.csv", index_col=0)
training_data.head()

,sentence,difficulty,note_orthographe,lexical_complexite,char_length,word_length,type_token_ratio,sentence_length,avg_word_length,complexite_texte,...,DET,PRON,NUM,NOUN,INTJ,ADP,ADJ,VERB,PROPN,SCONJ
id,,,,,,,,,,,,,,,,,,,,,
0,Les coûts kilométriques réels peuvent diverger...,4,1.000000,0.194007,0.160077,0.140152,0.467105,0.140152,0.339713,0.244565,...,0.066667,0.000000,0.000000,0.311111,0.0,0.288889,0.066667,0.088889,0.0000,0.0
1,"Le bleu, c'est ma couleur préférée mais je n'a...",0,1.000000,0.082334,0.036990,0.041667,1.000000,0.041667,0.204545,0.086957,...,0.187500,0.125000,0.000000,0.125000,0.0,0.000000,0.000000,0.125000,0.0625,0.0
2,Le test de niveau en français est sur le site ...,0,0.769231,0.088078,0.039541,0.045455,0.826923,0.045455,0.195804,0.081522,...,0.200000,0.000000,0.000000,0.400000,0.0,0.266667,0.000000,0.066667,0.0000,0.0
3,Est-ce que ton mari est aussi de Boston?,0,1.000000,0.062664,0.022959,0.026515,1.000000,0.026515,0.193182,0.054348,...,0.000000,0.100000,0.000000,0.300000,0.0,0.100000,0.000000,0.100000,0.1000,0.1
4,"Dans les écoles de commerce, dans les couloirs...",2,1.000000,0.184993,0.130740,0.125000,0.602941,0.125000,0.288770,0.228261,...,0.095238,0.047619,0.047619,0.238095,0.0,0.261905,0.047619,0.095238,0.0000,0.0


**pre-trained model FlauBERT base in french**

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# pre-trained model
model_name = "flaubert/flaubert_base_cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

phrases = training_data['sentence'].tolist()

# Use the tokenizer to convert phrases into FlauBERT tokens
tokenized_phrases = tokenizer(phrases, padding=True, truncation=True, return_tensors="pt", max_length=128)

with torch.no_grad():
    embeddings = model(**tokenized_phrases)

bert_embeddings = embeddings.last_hidden_state


**PCA gives us the best results compared to t-SNE or UMAP**

In [4]:
from sklearn.decomposition import PCA
num_examples, seq_length, embedding_dim = bert_embeddings.shape
bert_embeddings_2d = bert_embeddings.reshape(num_examples, -1) 

# Apply PCA to two-dimensional embeddings
num_components = 50
pca = PCA(n_components=num_components)
reduced_embeddings = pca.fit_transform(bert_embeddings_2d)
del pca

train test split

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

training_data_x = training_data.drop(columns=["difficulty", "sentence"])
X_combined = np.concatenate((reduced_embeddings, training_data_x), axis=1)

X = X_combined  
y = training_data["difficulty"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Régression Logistique :

In [6]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression()

# Train
logistic_regression_model.fit(X_train, y_train)

# Predict on the test set
y_pred = logistic_regression_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Logistic Regression model:", accuracy)

# Delete the model to free up memory
del logistic_regression_model

Accuracy of the Logistic Regression model: 0.44479166666666664


/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM (Support Vector Machine) :

In [7]:
from sklearn.svm import SVC

# Create an SVM classification model
svm_model = SVC()

# Train the model on the training set
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the SVM model:", accuracy)

# Delete the model to free up memory
del svm_model

Accuracy of the SVM model: 0.4666666666666667


Arbres de Décision :

In [8]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classification model
decision_tree_model = DecisionTreeClassifier()

# Train the model on the training set
decision_tree_model.fit(X_train, y_train)

# Predict on the test set
y_pred = decision_tree_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Decision Tree model:", accuracy)

# Delete the model to free up memory
del decision_tree_model

Accuracy of the Decision Tree model: 0.34375


Forêts Aléatoires :

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classification model
random_forest_model = RandomForestClassifier()

# Train the model on the training set
random_forest_model.fit(X_train, y_train)

# Predict on the test set
y_pred = random_forest_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Random Forest model:", accuracy)

# Delete the model to free up memory
del random_forest_model

Accuracy of the Random Forest model: 0.43645833333333334


Réseaux de Neurones (utilisant scikit-learn) :

In [10]:
from sklearn.neural_network import MLPClassifier

# Create an MLP classification model
mlp_model = MLPClassifier()

# Train the model on the training set
mlp_model.fit(X_train, y_train)

# Predict on the test set
y_pred = mlp_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the MLP model:", accuracy)

# Delete the model to free up memory
del mlp_model

Accuracy of the MLP model: 0.3875


/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Réseaux de Neurones Récursifs - RNN

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create an MLP classification model
rnn_model = MLPClassifier(hidden_layer_sizes=(100,), activation='tanh', solver='adam', max_iter=1000, random_state=42)

# Train the model on the training set
rnn_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rnn = rnn_model.predict(X_test)

# Evaluate the model using accuracy_score
accuracy_rnn = accuracy_score(y_test, y_pred_rnn)
print("Accuracy of the MLP model:", accuracy_rnn)

# Evaluate the model using classification_report
report_rnn = classification_report(y_test, y_pred_rnn)
print("\nClassification Report for the MLP model:\n", report_rnn)

# Delete the model to free up memory
del rnn_model

Accuracy of the MLP model: 0.3416666666666667

Classification Report for the MLP model:
               precision    recall  f1-score   support

           0       0.54      0.55      0.54       166
           1       0.32      0.32      0.32       158
           2       0.27      0.24      0.26       166
           3       0.25      0.29      0.27       153
           4       0.24      0.24      0.24       152
           5       0.41      0.39      0.40       165

    accuracy                           0.34       960
   macro avg       0.34      0.34      0.34       960
weighted avg       0.34      0.34      0.34       960

